# Introduction

This notebook demonstrates the use of the pre-trained M3GNet model to perform structural relaxation, molecular dynamics and single-point calculations.

Author: Tsz Wai Ko (Kenko)
Email: t1ko@ucsd.edu


In [ ]:
from __future__ import annotations

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from pymatgen.core import Lattice, Structure
from pymatgen.io.ase import AseAtomsAdaptor

from matgl.apps.pes import Potential
from matgl.ext.ase import M3GNetCalculator, MolecularDynamics, Relaxer
from matgl.models._m3gnet import M3GNet

# M3GNet Relaxation

The pre-trained model is trained on the Materials Project MP-2021.2.8 database.


In [ ]:
# load the pre-trained M3GNet model. By default it is the model trained to MP-2021.2.8 database.
model, d = M3GNet.load("pretrained_models/M3GNet-MP-2021.2.8-PES", include_json=True)
metadata = d["metadata"]
data_std = metadata["data_std"]
data_mean = metadata["data_mean"]
element_refs = metadata["element_refs"]
# create the potential class
ff = Potential(model, data_std=data_std, element_refs=element_refs)
# create the ase adaptor
ase_adaptor = AseAtomsAdaptor()
# create the M3GNet Relaxer
relaxer = Relaxer(potential=ff)
# This is the structure obtained from the Materials Project.
struct = Structure.from_spacegroup("Pm-3m", Lattice.cubic(4.1437), ["Cs", "Cl"], [[0, 0, 0], [0.5, 0.5, 0.5]])
relax_results = relaxer.relax(struct, fmax=0.01)
# extract results
final_structure = relax_results["final_structure"]
final_energy = relax_results["trajectory"].energies[-1]
# print out the final relaxed structure and energy
print(final_structure, final_energy)

/Users/kenko/miniconda3/envs/pythonProject/lib/python3.10/site-packages/dgl/backend/pytorch/tensor.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  return th.as_tensor(data, dtype=dtype)
/Users/kenko/miniconda3/envs/pythonProject/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/kenko/PycharmProjects/matgl/matgl/utils/maths.py:261: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or source

Full Formula (Cs1 Cl1)
Reduced Formula: CsCl
abc   :   4.210124   4.210124   4.210124
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cs    0    0    0
  1  Cl    0.5  0.5  0.5 [-6.5425954]


# M3GNet Molecular Dynamics

The molcular dynamic simulations were perform using M3GNet with ASE-interface.


In [ ]:
# create ase atom object
atoms = ase_adaptor.get_atoms(final_structure)
# initialize the velocity according to Maxwell Boltzamnn distribution
MaxwellBoltzmannDistribution(atoms, temperature_K=300)
# create the MD drive class
MD_driver = MolecularDynamics(atoms, potential=ff, temperature=300, logfile="md_trial.log")
# Run
MD_driver.run(100)
print(f"The potential energy of CsCl at 300 K after 100 STEP is {float(atoms.get_potential_energy()):.3f} eV.")

The potential energy of CsCl at 300 K after 100 STEP is -6.464 eV.


# M3GNet Calculator

Perform a single-point calculation for final structure using M3GNetCalculator.


In [ ]:
# define the M3GNet calculator
calc = M3GNetCalculator(ff)
# set up the calculator for atoms object
atoms.set_calculator(calc)
print(f"The calculated potential energy is {float(atoms.get_potential_energy()):.3f} eV.")

The calculated potential energy is -6.464 eV.
